In [1]:
# coding:utf-8

import pandas as pd
import numpy as np
import os,sys
from multiprocessing import Pool
import matplotlib.pyplot as plt

In [2]:
baseline = pd.read_csv('../datas/baseline_1part456')
baseline.head()

,device_city,device_cnt,device_cnt_0,device_cnt_1,device_cnt_1rate,device_cnt_rate,device_cnt_scan0,device_cnt_scan0rate,device_cnt_scan1,device_cnt_sec0,...,iptype_cnt_sec1,iptype_device,iptype_diff1,iptype_diff2,iptype_diff3,iptype_from,iptype_id,iptype_time_max,iptype_time_mean,iptype_time_min
0,1,5,0,5,0.833333,0.000000,5,0.833333,0,5,...,0,1,62724.0,NaN,NaN,1,1,30023.0,30023.000000,30023.0
1,1,6,1,5,0.714286,0.142857,6,0.857143,0,6,...,0,1,12.0,62736.0,NaN,1,1,30023.0,21576.500000,13130.0
2,1,38,0,38,0.974359,0.000000,38,0.974359,0,38,...,0,2,165.0,587.0,3008.0,2,1,10000.0,2458.843750,1708.0
3,2,74,11,63,0.840000,0.146667,74,0.986667,0,74,...,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN
4,278,12785,5833,6952,0.543720,0.456202,12674,0.991240,111,12785,...,0,1,63.0,125.0,125.0,1,1,4112.0,583.595428,81.0


In [4]:
baseline = baseline.set_index('idx')
baseline.head()

,device_city,device_cnt,device_cnt_0,device_cnt_1,device_cnt_1rate,device_cnt_rate,device_cnt_scan0,device_cnt_scan0rate,device_cnt_scan1,device_cnt_sec0,...,iptype_cnt_sec1,iptype_device,iptype_diff1,iptype_diff2,iptype_diff3,iptype_from,iptype_id,iptype_time_max,iptype_time_mean,iptype_time_min
idx,,,,,,,,,,,,,,,,,,,,,
101421,1,5,0,5,0.833333,0.000000,5,0.833333,0,5,...,0,1,62724.0,NaN,NaN,1,1,30023.0,30023.000000,30023.0
101422,1,6,1,5,0.714286,0.142857,6,0.857143,0,6,...,0,1,12.0,62736.0,NaN,1,1,30023.0,21576.500000,13130.0
101423,1,38,0,38,0.974359,0.000000,38,0.974359,0,38,...,0,2,165.0,587.0,3008.0,2,1,10000.0,2458.843750,1708.0
101424,2,74,11,63,0.840000,0.146667,74,0.986667,0,74,...,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN
101425,278,12785,5833,6952,0.543720,0.456202,12674,0.991240,111,12785,...,0,1,63.0,125.0,125.0,1,1,4112.0,583.595428,81.0


In [5]:
t_login = pd.read_csv('../datas/t_login.csv')

t_login = t_login[t_login['time']>='2015-03-01 00:00:00']


t_login['timestamp_online'] = t_login['timestamp'] + t_login['timelong']
t_login['result'] = t_login['result'].map(lambda x: x == 1 and 1 or -1)

t_login = t_login.sort_values('timestamp') \
                .reset_index(drop=True)


dtt = t_login[t_login['time']>='2015-04-01 00:00:00']
dtt.head()

,log_id,timelong,device,log_from,ip,city,result,timestamp,type,id,is_scan,is_sec,time,timestamp_online
101421,1.442511e+16,13130.0,812921,1,385900,217,-1,1.427818e+09,2,111553,False,False,2015-04-01 00:00:04,1.427831e+09
101422,4.721558e+14,24690.0,812921,1,385900,217,1,1.427818e+09,2,111553,False,False,2015-04-01 00:00:16,1.427842e+09
101423,1.939157e+15,1790.0,412258,1,194231,211,1,1.427818e+09,3,1195,False,False,2015-04-01 00:00:29,1.427819e+09
101424,2.886586e+16,7000.0,456923,2,1553851,243,1,1.427818e+09,3,121210,False,False,2015-04-01 00:00:33,1.427825e+09
101425,4.951091e+15,652.0,835072,1,359080,194,-1,1.427818e+09,1,74386,False,False,2015-04-01 00:00:49,1.427818e+09


In [6]:
data = dtt.merge(baseline,left_index=True,right_index=True)
data.head()

,log_id,timelong,device,log_from,ip,city,result,timestamp,type,id,...,iptype_cnt_sec1,iptype_device,iptype_diff1,iptype_diff2,iptype_diff3,iptype_from,iptype_id,iptype_time_max,iptype_time_mean,iptype_time_min
101421,1.442511e+16,13130.0,812921,1,385900,217,-1,1.427818e+09,2,111553,...,0,1,62724.0,NaN,NaN,1,1,30023.0,30023.000000,30023.0
101422,4.721558e+14,24690.0,812921,1,385900,217,1,1.427818e+09,2,111553,...,0,1,12.0,62736.0,NaN,1,1,30023.0,21576.500000,13130.0
101423,1.939157e+15,1790.0,412258,1,194231,211,1,1.427818e+09,3,1195,...,0,2,165.0,587.0,3008.0,2,1,10000.0,2458.843750,1708.0
101424,2.886586e+16,7000.0,456923,2,1553851,243,1,1.427818e+09,3,121210,...,0,0,NaN,NaN,NaN,0,0,NaN,NaN,NaN
101425,4.951091e+15,652.0,835072,1,359080,194,-1,1.427818e+09,1,74386,...,0,1,63.0,125.0,125.0,1,1,4112.0,583.595428,81.0


In [7]:
data.to_csv('../datas/baseline_feas_train',index=None)